In [260]:
# import dependencies 
import math 
import numpy as np
from itertools import combinations

In [261]:
def calculate_proportion(project, project_subset, funding_score):
    """
    Calculate the proportion of the funding score of a project relative to the total funding scores of all projects in the subset.

    Args:
    - project: The project for which the proportion is being calculated.
    - project_subset: The subset of projects among which the proportion is being calculated.
    - funding_score: Dictionary containing funding scores of all projects.

    Returns:
    - Proportion of the funding score of the project relative to the total funding scores of all projects in the subset.
    """
    total_subset_score = sum(funding_score[q] for q in project_subset)
    project_score = funding_score[project]
    proportion = project_score / total_subset_score
    
    return proportion

In [262]:
def change_set(p, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds):
    """
    A subroutine to make sure a project is in the appropriate set
    """
    for s in above_max, at_max, between, below_min, zero:
        s.discard(p)
    
    # Determine which set 'p' belongs to based on its funding value
    if funding[p] > max_funds[p]:
        above_max.add(p)
    elif funding[p] == max_funds[p]:
        at_max.add(p)
    elif funding[p] >= min_funds[p]:
        between.add(p)
    elif funding[p] > 0:
        below_min.add(p)
    else:
        zero.add(p)
    
    # Check that funding value of 'p' is non-negative
    assert funding[p] >= 0


In [263]:
def distribute_excess(project_set, funding, excess, fundingScore, above_max, at_max, between, below_min, zero, max_funds, min_funds):
    for p in project_set:
        funding[p] += excess * calculate_proportion(p, project_set, fundingScore)
        change_set(project_set, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds)

In [264]:
def find_worst_performing_projects(below_min, funding, min_funds):
    """
    Find the set of worst-performing projects with non-zero funding.

    Args:
    - below_min (set): Set of project names below minimum performance.
    - funding (dict): Dictionary mapping project names to their funding values.
    - mins (dict): Dictionary mapping project names to their minimum performance values.

    Returns:
    - set: Set of worst-performing projects with non-zero funding.
    """
    min_funding = min(funding[p] / min_funds[p] for p in below_min)
    
    return {p for p in below_min if funding[p] / min_funds[p] == min_funding}

In [265]:
"""
projects = ['0', '1']
fundingScore = {'0': 2, '1': 3}
min_funds = {'0': 30000, '1': 70000}
max_funds = {'0': 30000, '1': 70000}
total_budget = 100000

above_max, at_max, between, below_min, zero = set(), set(), set(), set(), set()
"""

"\nprojects = ['0', '1']\nfundingScore = {'0': 2, '1': 3}\nmin_funds = {'0': 30000, '1': 70000}\nmax_funds = {'0': 30000, '1': 70000}\ntotal_budget = 100000\n\nabove_max, at_max, between, below_min, zero = set(), set(), set(), set(), set()\n"

In [266]:
""" 
# Set initial values for funding and excess
funding = {p: total_budget * calculate_proportion(p, projects, fundingScore) for p in projects}
print(funding)
"""

' \n# Set initial values for funding and excess\nfunding = {p: total_budget * calculate_proportion(p, projects, fundingScore) for p in projects}\nprint(funding)\n'

In [267]:
"""
for p in projects:
    change_set(p, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds)

print("above_max:", above_max)
print("at_max:", at_max)
print("between:", between)
print("below_min:", below_min)
print("zero:", zero)
"""

'\nfor p in projects:\n    change_set(p, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds)\n\nprint("above_max:", above_max)\nprint("at_max:", at_max)\nprint("between:", between)\nprint("below_min:", below_min)\nprint("zero:", zero)\n'

In [268]:
"""
worst_projects = find_worst_performing_projects(below_min, funding, min_funds)
print(worst_projects)
"""

'\nworst_projects = find_worst_performing_projects(below_min, funding, min_funds)\nprint(worst_projects)\n'

In [269]:
 """
def main_loop(above_max, below_min, at_max, between, funding, max_funds, min_funds):
    Perform the main loop of the project funding redistribution algorithm until convergence.

    Args:
    - above_max (set): Set of project names above maximum performance.
    - below_min (set): Set of project names below minimum performance.
    - between (set): Set of project names between minimum and maximum performance.
    - funding (dict): Dictionary mapping project names to their funding values.
    - maxs (dict): Dictionary mapping project names to their maximum performance values.
    - change_set (function): Function to change the project set.
    - distribute_excess (function): Function to distribute excess funding among projects.
    excess = 0

    while len(above_max) + len(below_min) > 0:

        # If any projects are above their maximum, remove the excess and redistribute it
        while len(above_max) > 0:
            for p in above_max.copy():
                # Iterate through a copy because changing a set's size while iterating through it is not allowed
                excess += funding[p] - max_funds[p]
                funding[p] = max_funds[p]
                change_set(p, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds)

            if len(below_min.union(between)) > 0:
                distribute_excess(below_min.union(between).copy(), funding, excess, fundingScore, above_max, at_max, between, below_min, zero, max_funds, min_funds)
                excess = 0

        # If there is still at least one project below its minimum level of funding, remove the worst performing one and redistribute its funding.
        if len(below_min) > 0:
            for p in find_worst_performing_projects(below_min, funding, min_funds).copy():
                excess += funding[p]
                funding[p] = 0
                change_set(p, funding, above_max, at_max, between, below_min, zero, max_funds, min_funds)

            if len(below_min.union(between)) > 0:
                distribute_excess(below_min.union(between).copy(), funding, excess, fundingScore, above_max, at_max, between, below_min, zero, max_funds, min_funds)
                excess = 0

        # After each pass in this while loop, len(above_max) is 0 and len(below_min) is at least 1 smaller than it was before.
        # So we are guaranteed to terminate.

    return funding, excess
 """

"\ndef main_loop(above_max, below_min, at_max, between, funding, max_funds, min_funds):\n   Perform the main loop of the project funding redistribution algorithm until convergence.\n\n   Args:\n   - above_max (set): Set of project names above maximum performance.\n   - below_min (set): Set of project names below minimum performance.\n   - between (set): Set of project names between minimum and maximum performance.\n   - funding (dict): Dictionary mapping project names to their funding values.\n   - maxs (dict): Dictionary mapping project names to their maximum performance values.\n   - change_set (function): Function to change the project set.\n   - distribute_excess (function): Function to distribute excess funding among projects.\n   excess = 0\n\n   while len(above_max) + len(below_min) > 0:\n\n       # If any projects are above their maximum, remove the excess and redistribute it\n       while len(above_max) > 0:\n           for p in above_max.copy():\n               # Iterate thro

In [270]:
## result_funding, result_excess = main_loop(above_max, below_min, at_max, between, funding, max_funds, min_funds)
## print(result_funding, result_excess)

In [271]:
def scale_funding(projects, fundingScore, mins, maxs, total_budget):
  '''
  input:
    projects - list (int) of projects
    fundingScore - dictionary indexed by projects. fundingScore[p] = output of COCM for project p
    mins - dictionary indexed by projects. mins[p] = minimum viable funding for project p
    maxs - dictionary indexed by projects. mins[p] = maximum viable funding for project p
    total budget - float. total amount of funding to give out.

  output:
    funding - dictionary indexed by projects. funding[p] = actual amount of funding to give to project p. funding[p] is either 0, or in between min[p] and max[p]
    excess - float. amount of unused funding (greater than 0 if, for example, total_budget is bigger than the sum of all maximum budgets)
  '''

  # subroutine: given a project and a subset of projects, what is fundingScore[p] as a percentage of the scores among projects in that set?
  proportion = lambda p,x : fundingScore[p] / sum(fundingScore[q] for q in x)
  
  for p in projects:
            print(f"Proportion for project {p}: {proportion(p, projects)}")

  # set initial values for funding and excess
  funding = {p: total_budget * proportion(p, projects) for p in projects}
   
  print(f"Project funding: {funding}")
  
  excess = 0

  # as we move funding around, we'll use these sets to keep track of how much funding different projects have
  above_max, at_max, between, below_min, zero = set(), set(), set(), set(), set()

  # a subroutine to make sure a project is in the appropriate set
  def change_set(p):
    for s in above_max, at_max, between, below_min, zero:
      s.discard(p)
    if   funding[p] > maxs[p] : above_max.add(p)
    elif funding[p] == maxs[p]: at_max.add(p)
    elif funding[p] >= mins[p]: between.add(p)
    elif funding[p] > 0       : below_min.add(p)
    else                      : zero.add(p)
    assert funding[p] >= 0

  # run the above subroutine project to put every project in the right set
  for p in projects: change_set(p)

  # a subroutine to take the current excess and redistribute it among projects in the set s
  def distribute_excess(s):
    for p in s:
      funding[p] += excess * proportion(p, s)
      change_set(p)

  # a subroutine to find the set of worst-performing projects with non-zero funding
  min_funding = lambda: {p for p in below_min if funding[p] / mins[p] == min(funding[q] / mins[q] for q in below_min)}

  # main loop
  while len(above_max) + len(below_min) > 0:

    # if any projects are above their maximum, remove the excess and redistribute it
    while len(above_max) > 0:
      for p in above_max.copy():
        # iterate through a copy because changing a set's size while you're iterating through it is not allowed
        excess += funding[p] - maxs[p]
        funding[p] = maxs[p]
        change_set(p)

      if len(below_min.union(between)) > 0:
        distribute_excess(below_min.union(between).copy())
        excess = 0

    # if there is still at least one project below its minimum level of funding, remove the worst performing one and redistribute its funding.
    if len(below_min) > 0:

      for p in min_funding().copy():
        excess += funding[p]
        funding[p] = 0
        change_set(p)

      if len(below_min.union(between)) > 0:
        distribute_excess(below_min.union(between).copy())
        excess = 0

    # after each pass in this while loop, len(above_max) is 0 and len(below_min) is at least 1 smaller than it was before.
    # So we are guaranteed to terminate.

  return funding, excess

In [272]:
projects = ['0', '1']
fundingScore = {'0': 2, '1': 3}
min_funds = {'0': 30000, '1': 70000}
max_funds = {'0': 50000, '1': 80000}
total_budget = 100000

print(fundingScore)
print(min_funds)
print(max_funds)

f, e = scale_funding(projects, fundingScore, min_funds, max_funds, total_budget)
print(f)
print(e)
print(sum(f[p] for p in projects) + e)

{'0': 2, '1': 3}
{'0': 30000, '1': 70000}
{'0': 50000, '1': 80000}
Proportion for project 0: 0.4
Proportion for project 1: 0.6
Project funding: {'0': 40000.0, '1': 60000.0}
{'0': 50000, '1': 0}
50000.0
100000.0


In [273]:
projects = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
fundingScore = {'0': 1, '1': 1.5, '2': 2, '3': 2.5,'4': 3, '5': 3.5,'6': 4, '7': 4.5,'8': 5, '9': 5.5,'10': 6, '11': 6.5,'12': 7, '13': 7.5,'14': 8, '15': 8.5}
min_funds = {'0': 30000, '1': 70000, '2': 30000, '3': 70000, '4': 30000, '5': 70000,'6': 30000, '7': 70000,'8': 30000, '9': 70000,'10': 30000, '11': 70000,'12': 30000, '13': 70000,'14': 30000, '15': 70000}
max_funds = {'0': 30000, '1': 70000, '2': 30000, '3': 70000, '4': 30000, '5': 70000,'6': 30000, '7': 70000,'8': 30000, '9': 70000,'10': 30000, '11': 70000,'12': 30000, '13': 70000,'14': 30000, '15': 70000}
total_budget = 100000

print(fundingScore)
print(min_funds)
print(max_funds)

f, e = scale_funding(projects, fundingScore, min_funds, max_funds, total_budget)
print(f)
print(e)
print(sum(f[p] for p in projects) + e)

{'0': 1, '1': 1.5, '2': 2, '3': 2.5, '4': 3, '5': 3.5, '6': 4, '7': 4.5, '8': 5, '9': 5.5, '10': 6, '11': 6.5, '12': 7, '13': 7.5, '14': 8, '15': 8.5}
{'0': 30000, '1': 70000, '2': 30000, '3': 70000, '4': 30000, '5': 70000, '6': 30000, '7': 70000, '8': 30000, '9': 70000, '10': 30000, '11': 70000, '12': 30000, '13': 70000, '14': 30000, '15': 70000}
{'0': 30000, '1': 70000, '2': 30000, '3': 70000, '4': 30000, '5': 70000, '6': 30000, '7': 70000, '8': 30000, '9': 70000, '10': 30000, '11': 70000, '12': 30000, '13': 70000, '14': 30000, '15': 70000}
Proportion for project 0: 0.013157894736842105
Proportion for project 1: 0.019736842105263157
Proportion for project 2: 0.02631578947368421
Proportion for project 3: 0.03289473684210526
Proportion for project 4: 0.039473684210526314
Proportion for project 5: 0.046052631578947366
Proportion for project 6: 0.05263157894736842
Proportion for project 7: 0.05921052631578947
Proportion for project 8: 0.06578947368421052
Proportion for project 9: 0.07236